## Baseline solution

In this notebook , we will create a baseline solution to our semantic segmentation problem . To iterate fast a notebook is a handy solution. We will then refactor this code into a script to be able to use hyperparameter sweeps. 

In [1]:
import wandb
import pandas as pd
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback

import params
from util import get_predictions, create_iou_table, MIOU, BackgroundIOU, \
                  RoadIOU, TrafficLightIOU, TrafficSignIOU, PersonIOU, VehicleIOU, BicycleIOU
     

In [2]:
train_config = SimpleNamespace(
    framework="fastai",
    img_size=(180, 320),
    batch_size=8,
    augment=True, # use data augmentation
    epochs=10, 
    lr=2e-3,
    pretrained=True,  # whether to use pretrained encoder
    seed=42,
)

We are setting seed for reproducibility.

In [3]:
set_seed(train_config.seed, reproducible=True)

In [4]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="training", config=train_config)
     

wandb: Currently logged in as: twelvve. Use `wandb login --relogin` to force relogin


As usual, we will use W&B Artifacts to track the lineage of our models.

In [5]:
processed_data_at = run.use_artifact(f'{params.PROCESSED_DATA_AT}:latest')
processed_dataset_dir = Path(processed_data_at.download())
df = pd.read_csv(processed_dataset_dir / 'data_split.csv')

wandb: Downloading large artifact bdd_simple_1k_split:latest, 813.25MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:0:29.7


We will not use the hold out dataset stage at this moment. is_valid column will tell our trainer how we want to split data between training and validation.

In [6]:
df = df[df.Stage != 'test'].reset_index(drop=True)
df['is_valid'] = df.Stage == 'valid'

In [7]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"
     

We will use fastai's DataBlock API to feed data into model training and validation.

In [8]:
# assign paths
df["image_fname"] = [processed_dataset_dir/f'images/{f}' for f in df.File_Name.values]
df["label_fname"] = [label_func(f) for f in df.image_fname.values]

In [9]:
def get_data(df, bs=4, img_size=(180, 320), augment=True):
    block = DataBlock(blocks=(ImageBlock, MaskBlock(codes=params.BDD_CLASSES)),
                  get_x=ColReader("image_fname"),
                  get_y=ColReader("label_fname"),
                  splitter=ColSplitter(),
                  item_tfms=Resize(img_size),
                  batch_tfms=aug_transforms() if augment else None,
                 )
    return block.dataloaders(df, bs=bs)

We are using wandb.config to track our training hyperparameters.

In [10]:
config = wandb.config

In [11]:
dls = get_data(df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

We will use intersection over union metrics: mean across all classes (MIOU) and IOU for each class separately. Our model will be a unet based on pretrained resnet18 backbone.

In [12]:
metrics = [MIOU(), BackgroundIOU(), RoadIOU(), TrafficLightIOU(), TrafficSignIOU(), PersonIOU(), VehicleIOU(), BicycleIOU()]
learn = unet_learner(dls, arch=resnet18, pretrained=config.pretrained, metrics=metrics)

/home/l3gion/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/l3gion/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/l3gion/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In fastai we already have a callback that integrates tightly with W&B, we only need to pass the WandbCallback to the learner and we are ready to go. The callback will log all the useful variables for us. For example, whatever metric we pass to the learner will be tracked by the callback.

In [13]:
callbacks = [
    SaveModelCallback(monitor='miou'),
    WandbCallback(log_preds=False, log_model=True)
]

lets' train our model

In [14]:
learn.fit_one_cycle(config.epochs, config.lr, cbs=callbacks)

epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.499578,0.364236,0.310812,0.850326,0.720663,0.000000,0.000000,0.000000,0.604696,0.000000,15:08
1,0.421260,0.718704,0.209882,0.738757,0.122866,0.000000,0.000000,0.000000,0.607549,0.000000,14:49
2,0.354325,0.293813,0.332081,0.885431,0.777302,0.000000,0.000000,0.000000,0.661834,0.000000,14:40
3,0.305278,0.284105,0.337113,0.890631,0.796875,0.000000,0.000000,0.000000,0.672286,0.000000,14:38
4,0.274761,0.277250,0.345085,0.903040,0.824483,0.000000,0.000000,0.000000,0.688074,0.000000,14:39
5,0.254189,0.259829,0.346546,0.900076,0.821107,0.000000,0.000000,0.000000,0.704638,0.000000,14:24
6,0.226936,0.234578,0.359877,0.916662,0.837949,0.029374,0.000000,0.000000,0.735151,0.000000,14:20
7,0.203871,0.250662,0.360631,0.911581,0.829415,0.040405,0.000000,0.000000,0.743019,0.000000,14:19
8,0.196916,0.226973,0.374627,0.921230,0.847536,0.103665,0.000000,0.000000,0.749958,0.000000,14:15
9,0.179295,0.227312,0.370638,0.920992,0.848762,0.074664,0.000000,0.000000,0.750046,0.000000,14:15


Better model found at epoch 0 with miou value: 0.31081200611485643.
Better model found at epoch 2 with miou value: 0.3320810562349588.
Better model found at epoch 3 with miou value: 0.33711306523300116.
Better model found at epoch 4 with miou value: 0.34508527376916237.
Better model found at epoch 5 with miou value: 0.3465458256791535.
Better model found at epoch 6 with miou value: 0.3598765198340405.
Better model found at epoch 7 with miou value: 0.3606314944231466.
Better model found at epoch 8 with miou value: 0.3746271114287773.


We will log a table with model predictions and ground truth to W&B, so that we can do error analysis in the W&B dashboard.

In [15]:
samples, outputs, predictions = get_predictions(learn)
table = create_iou_table(samples, outputs, predictions, params.BDD_CLASSES)
wandb.log({"pred_table":table})

We are reloading the model from the best checkpoint at the end and saving it. To make sure we track the final metrics correctly, we will validate the model again and save the final loss and metrics to wandb.summary.

In [16]:
scores = learn.validate()
metric_names = ['final_loss'] + [f'final_{x.name}' for x in metrics]
final_results = {metric_names[i] : scores[i] for i in range(len(scores))}
for k,v in final_results.items(): 
    wandb.summary[k] = v

In [17]:
wandb.finish()

background_iou,▅▁▇▇▇▇████
bicycle_iou,▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
lr_1,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
lr_2,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
miou,▅▁▆▆▇▇▇▇██
mom_0,██▇▆▅▄▃▂▁▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██████
